# Prelab 10: Preparing to reanalyze your Lab 09 data

In [ ]:
%reset -f
import data_entry2
import numpy as np
import matplotlib.pyplot as plt

In Lab 09, it is likely that you encountered some unexpected behaviour in your data due to background radiation, meaning radiation coming from something other than the source we provided you with, such as cosmic rays. And this background radiation likely posed significant challenges when trying to fit a linear model to `Ln(R)` data. 

In Lab 10, you will reanlyze your Lab 09 data with a new nonlinear model that accounts for this background radiation. Additionally, you will collect the additional data, mostly at larger thicknesses of shielding, to help you extract the background radiation count-rate parameter from your nonlinear model with a high level of confidence.

Unfortunately, we don't have nice analytic solutions that can handle the new nonlinear model we will be using. As a result, we will use a new `fit_plot` widget that works with our linear model to help us find our best-fit parameters through chi-squared minimization. Additionally, in this prelab, we will modify some of our previous fitting and plotting code to see how we can account for this nonlinear model.

## 1. A hypothetical data set that shows a similar type of behaviour to the Lab 09 data

Below we introduce a hypothetical data set that was generated computationally to represent data taken with equipment that will give somewhat different numerical results to those that we would expect to achieve in our lab with our equipment. 

**The idea is that these data will provide graphs with the same types of shapes that we expect to see with the real lab equipment, but the values we will extract from the nonlinear fits of these hypothetical data WILL NOT MATCH those from the real lab equipment.**

In the code cell below, we load and plot the hypothetical data set. Have a look at these plots and then answer the Your Turn question.

In [ ]:
# Run me to load and plot our hypothetical data set

# Our data arrays

x_h = np.array([
        0., 2., 4., 6., 8., 10., 12., 14., 16., 18., 20., 22., 24., 26.
        ])  # number sheets of plastic shielding
Rate_h = np.array([
        18.54237288, 9.06666667, 4.54237288, 2.55, 1.31666667,
        0.76666667, 0.48333333, 0.44262295, 0.51666667, 0.35483871,
        0.48387097, 0.45070423, 0.68333333, 0.47540984
        ])  # counts per second
dRate_h = np.array([
        0.59006424, 0.38873013, 0.27746959, 0.20615528, 0.14813657,
        0.11303883, 0.08975275, 0.08518283, 0.09279607, 0.07565187,
        0.08834235, 0.079674, 0.10671874, 0.08828139
        ])  # counts per second

# Convert rates to log(Rates) for the semi log plot

# See Prelab 09 for a reminder of where these equations came from
logRate_h = np.log(Rate_h)
dlogRate_h = dRate_h / Rate_h

# Plot the untransformed data

xdata = x_h
ydata = Rate_h
dydata = dRate_h

graph_title = "Untransformed hypothetical count-rate data"
x_label = "Shielding Thickness (x, number of plastic sheets)"
y_label = "Count Rate (R, counts/second)"

plt.figure()
plt.errorbar(
        xdata, ydata, dydata, marker='.',
        linestyle='', label="Hypothetical data (Untransformed)"
        )
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.title(graph_title)
plt.legend()
plt.grid(True)
plt.show()

# Plot the linearized data

xdata = x_h
ydata = logRate_h
dydata = dlogRate_h

graph_title = "Linearized hypothetical count-rate data (semilog plot)"
x_label = "Shielding Thickness (x, number of plastic sheets)"
y_label = "Ln(R)"

plt.figure()
plt.errorbar(
        xdata, ydata, dydata, marker='.',
        linestyle='', label="Hypethetical data (Ln transformed)"
        )
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.title(graph_title)
plt.legend()
plt.grid(True)
plt.show()

### Your Turn #1

In the top plot (the untransformed hypothetical data) you can see that the count-rate never decays to 0. These counts are coming from background radiation and are not affected by introducing more sheets of our shielding. Similarly, without background radiation, we expect our bottom plot (the semilog plot) to be nice and straight, with $\mu = -m$, meaning the attentuation coefficient $\mu$ should be the negative of the slope of this graph. **However,** what we see is that our linearized data start nice and linear but start to plateau around 12 sheets.

Based on these graphs, or looking directly at the values in the `Rate_h` vector, what would you estimate the background radiation count-rate to be?

##### **Answer**

The background count-rate appears to be ~ 0.5 counts/second.

* You can see this directly in the top plot or the values in the `Rate_h` vector, where the values seem to be scatter around, but near to 0.5 counts/second for shielding thicknesses of 12+.
* We can also see in the semilog plot that the data plateau at $\ln(R) \approx -0.7$.
* Since $\ln(0.5) \approx -0.7$, we can see that both graphs lead us to the same conclusion. 

## 1.1 Fitting the entire linearized data set

Based on the fact that the linearized hypothetical count-rate data do not look linear, we know that it will go poorly if we try to fit a linear $y=mx+b$ model to this data set. But let's perform this fit to demonstrate how poor of a fit it is.

Let's grab a copy of our "2-parameter analytic best-fit solution" code from Prelab 09 and update the variables, labels and titles, as needed. Below we perform the fit

In [ ]:
# 2-parameter analytic best-fit solution and plotting from Prelab 09.

# Define the variables we will be plotting.
xdata = x_h
ydata = logRate_h
dydata = dlogRate_h

# Labels and titles
data_label = "Hypothetical Data (Ln transformed)"
model_label = "y = mx + b"
graph_title = "The best-fit line using the two-parameter analytic equations"
x_label = "Shielding Thickness (x, number of plastic sheets)"
y_label = "Ln(R)"
residuals_y_label = "Residual = data - model (units)"

# Find the best 2-parameter model corresponding to the minimized chi-squared.

# Calculate Z
Z = (
    np.sum(1 / dydata**2) * np.sum(xdata**2 / dydata**2)
    - np.sum(xdata / dydata**2)**2
)

# Calculate best fit slope, m
m = 1 / Z * (
    np.sum(1 / dydata**2) * np.sum(xdata * ydata / dydata**2)
    - np.sum(xdata / dydata**2) * np.sum(ydata / dydata**2)
)

# Calculate best fit y-intercept, b
b = 1 / Z * (
    np.sum(xdata**2 / dydata**2) * np.sum(ydata / dydata**2)
    - np.sum(xdata / dydata**2) * np.sum(xdata * ydata / dydata**2)
)

# Calculate uncertainty in best fit slope, dm
dm = np.sqrt(1 / Z * np.sum(1 / dydata**2))

# Calculate uncertainty in best fit y-intercept, db
db = np.sqrt(1 / Z * np.sum(xdata**2 / dydata**2))

# Print the best fit slope and uncertainty
print("Best fit slope, m = ", m, "±", dm)

# Print the best fit y-intercept and uncertainty
print("Best fit y-intercept, b = ", b, "±", db)


# Construct the model for plotting and calculating residuals

ymodel = m * xdata + b  # best fit model
res = ydata - ymodel  # calculate residuals (best fit)
wres2 = (res / dydata)**2  # weighted residuals squared


# Calculate chi-squared
    
N = len(xdata)  # number of data points
P = 2  # number of parameters in y = mx + b
chi2 = np.sum(wres2) / (N - P)  # calculate chi-squared
print(f"chi2 = {chi2:.4f}")

# Plot data and fit

plt.figure()
plt.errorbar(
    xdata, ydata, dydata, marker='.', linestyle='',
    color='b', label=data_label
    )
plt.plot(xdata, ymodel, color='r', label=model_label)
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.title(graph_title)
plt.legend()
plt.grid()
plt.show()

# Plot residuals for the best fit

plt.figure()
plt.errorbar(xdata, res, dydata, marker='.', linestyle='', color='b')
plt.hlines(
    y=0, xmin=np.min(xdata), xmax=np.max(xdata), color='k'
    )  # draw axis at y = 0
plt.xlabel(x_label)
plt.ylabel(residuals_y_label)
plt.title(f'Residuals plot (best fit, $\chi^2$={chi2:.4f})')
plt.grid()
plt.show

### Your Turn #2

Based on the plots above and the chi-squared value displayed, what are your conclusions about how well the $R(x) = R_0 \, e^{-\mu x}$ model fits these data? 

Keep in mind that for the semilog plot, this model predicts a linear graph of the form $\text{Ln}(R(x)) = -\mu x + \text{Ln}(R_0).$

##### **Answer**

We get $\chi^2=42.4$ and residuals with an extremely obvious trend showing that the linear model does not describe the data very well at all. This is a very poor fit, as expected.

## 1.2 Fitting only the early linear portion of this data set

We can see in the scatter and residuals plots above that the data look to be reasonably linear up to and including a shielding thickness of 10 plastic sheets. Without our understanding that there is also a background radiation count-rate in every one of these measurements, our understanding of the situation is that $x > 10$ corresponds to count-rates that are dominated by background radiation. So let's truncate our data set to only include $x \le 10$ and re-run our fit.

Note that we use a special way of subsetting a python array called a mask when defining our variables: `xdata = x_h[x_h<=10]`. This says that our variable `xdata` will be an array that contains only the values from our `x_h` array that are less than or equal to `<= 10`. You do not need to know how to do this, but we point it out here for those curious to learn some new python skills.

When we re-run the fit using only those first six data points we get something that looks much more linear based on the scatter and residuals plots, and the fit has a chi-squared value of 0.78. We have a good fit.

In [ ]:
# 2-parameter analytic best-fit solution and plotting from Prelab 09

# Define the variables we will be plotting
xdata = x_h[x_h <= 10]
ydata = logRate_h[x_h <= 10]
dydata = dlogRate_h[x_h <= 10]

# Labels and titles
data_label = "Hypothetical Data"
model_label = "y = mx + b"
graph_title = "The best-fit line using the two-parameter analytic equations"
x_label = "Shielding Thickness (x, number of plastic sheets)"
y_label = "Ln(R)"
residuals_y_label = "Residual = data - model (units)"

# Find the best 2-parameter model corresponding to the minimized chi-squared

# Calculate Z
Z = (
    np.sum(1 / dydata**2) * np.sum(xdata**2 / dydata**2)
    - np.sum(xdata / dydata**2)**2
)

# Calculate best fit slope, m
m = 1 / Z * (
    np.sum(1 / dydata**2) * np.sum(xdata * ydata / dydata**2)
    - np.sum(xdata / dydata**2) * np.sum(ydata / dydata**2)
)

# Calculate best fit y-intercept, b
b = 1 / Z * (
    np.sum(xdata**2 / dydata**2) * np.sum(ydata / dydata**2)
    - np.sum(xdata / dydata**2) * np.sum(xdata * ydata / dydata**2)
)

# Calculate uncertainty in best fit slope, dm
dm = np.sqrt(1 / Z * np.sum(1 / dydata**2))

# Calculate uncertainty in best fit y-intercept, db
db = np.sqrt(1 / Z * np.sum(xdata**2 / dydata**2))

# Print the best fit slope and uncertainty
print("Best fit slope, m = ", m, "±", dm)

# Print the best fit y-intercept and uncertainty
print("Best fit y-intercept, b = ", b, "±", db)


# Construct the model for plotting and calculating residuals

ymodel = m * xdata + b  # best fit model
res = ydata - ymodel  # calculate residuals (best fit)
wres2 = (res / dydata)**2  # weighted residuals squared


# Calculate chi-squared
    
N = len(xdata)  # number of data points
P = 2  # number of parameters in y = mx + b
chi2 = np.sum(wres2) / (N - P)  # calculate chi-squared
print(f"chi2 = {chi2:.4f}")

# Plot data and fit

plt.figure()
plt.errorbar(
    xdata, ydata, dydata, marker='.',
    linestyle='', color='b', label=data_label
    )
plt.plot(xdata, ymodel, color='r', label=model_label)
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.title(graph_title)
plt.legend()
plt.grid()
plt.show()

# Plot residuals for the best fit

plt.figure()
plt.errorbar(xdata, res, dydata, marker='.', linestyle='', color='b')
plt.hlines(
    y=0, xmin=np.min(xdata), xmax=np.max(xdata), color='k'
    )  # draw axis at y = 0.
plt.xlabel(x_label)
plt.ylabel(residuals_y_label)
plt.title(f'Residuals plot (best fit, $\chi^2$={chi2:.4f})')
plt.grid()
plt.show

### Your Turn #3

Based on the best-fit parameters listed above for our fit on our truncated data set, what value would we report for the attentuation coefficient, $\mu$?

##### **Answer**

 We would report our attenuation coefficient ($\mu = -m$) as $\mu = (0.3307 \pm 0.0087) \text{ sheets}^{-1}$.

## 2. Introducing the nonlinear model with a background radiation term

A quick recap to what we have done so far in this prelab. First, we recognized that background radiation was present in our hypothetical data set (like it was in your Lab 09 data). So we truncated our data so that we only included shielding thicknesses where the radiation from our source dominated our count-rate data. Or to put it another way, we excluded the data where the background was dominiting the count-rates.

In Lab 10, we introduce an alternate way to tackle this issue, which is to build the background radiation count-rate directly into our model. To do so, we introduce a new term, $R_b$, the background count-rate, and we write our model as

$$R(x) = R_0 \, e^{-\mu x} + R_b.$$

Like before, $\mu$ is the attenuation coefficient of the shielding, $x$ is the thickness of the shielding in number of sheets, $R(x)$ is the measured count-rate and $R_0$ is the count-rate with no shielding, as predicted by the model.

### Your Turn #4

Based on our best-fit parameters and other observations of the hypothetical data set from Section 1, we can make reasonable initial estimates for each of the model parameters in 

$$R(x) = R_0 \, e^{-\mu x} + R_b.$$

Below we list some reasonable initial estimates for each of these parameters. Try to determine where each of these values comes from and then check the answers to confirm.

##### **Parameter 1 estimate:** $R_b = 0.5 \text{ counts/second}.$ 
How did we estimate this value?

##### **Answer**

This value comes from our earliest observation in this prelab that the data plateau at a count-rate of approximately $0.5$ or $\ln(-0.7)$.

##### **Parameter 2 estimate:** $R_0 = 18.0 \text{ counts/second} = (18.5 - 0.5) \text{ counts/second}.$
How did we estimate this value?

##### **Answer**

Our count-rate data for shielding thickness = 0 is 18.54237288 counts/second. Given that ~ 0.5 counts/second of this count-rate are due to the background count-rate, that means out no-shielding count-rate from just from the source is ~ 18.5 - 0.5 = 18.0 counts/second.

##### **Parameter 3 estimate:** $\mu = 0.33 \text{ sheets}^{-1}$.
How did we estimate this value?

##### **Answer**

The attentuation coefficient can be estimated from the slope of the linear portion of our semilog plot. This is how you determined the attenuation coefficient in Lab 08.

## 2.1 Plotting the $R(x) = R_0 \, e^{-\mu x} + R_b$ model with the initial guesses

In the previous section we developed some initial guesses for the fitting parameters $R_0$, $\mu$ and $R_b$. In the figure below we plot the untransformed data and add the line for the $R(x) = R_0 \, e^{-\mu x} + R_b$ model using these initial guesses.

In [ ]:
# Initial guesses for model parameters

R0 = 18  # counts/s
mu = 0.33  # 1/sheets
Rb = 0.5  # counts/s


# Plot the untransformed data

xdata = x_h
ydata = Rate_h
dydata = dRate_h

# Find the x and y values for the model line
xmin = np.min(xdata)  # use the np.min function to find the smallest x-value
xmax = np.max(xdata)  # same for max
xpoints = np.linspace(xmin, xmax, 200)
ypoints = R0 * np.exp(-mu * xpoints) + Rb  # R0 * e^{-mu*x} + Rb model

graph_title = "Untransformed hypothetical count-rate data"
x_label = "Shielding Thickness (x, number of plastic sheets)"
y_label = "Count Rate (R, counts/second)"

plt.figure()
plt.errorbar(
    xdata, ydata, dydata, marker='.',
    linestyle='', label="Hypothetical data (Untransformed)"
    )
plt.plot(
    xpoints, ypoints, "r-",
    label="$R_0 \, e^{-\mu x} + R_b$ model using initial guesses"
    )
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.title(graph_title)
plt.legend()
plt.grid(True)
plt.show()


We can see in the figure above that, with some adjustments to our initial guesses of the fitting parameters, we could probably have good agreement between our model and data.

## 2.2 Plotting the $\text{Ln}(R(x)) = \text{Ln}(R_0 \, e^{-\mu x} + R_b)$ model with the initial guesses

Let's now look at how our model with initial guesses looks on our semi-log plot. To do so we use our $\text{Ln}$ transformed data and our $\text{Ln}$ transformed model, $\text{Ln}(R(x)) = \text{Ln}(R_0 \, e^{-\mu x} + R_b)$.

The code box below creates a scatter plot, which includes some additional annotations to help relate the fit parameters $R_0$, $\mu$ and $R_b$ to the shape of the model. Additionally, it creates a residuals plots, outputs the current values of the fit parameters, as well as calculating and outputting chi-squared. It is not expected that you need to be able to follow all of this code. Instead, have a look at the plots that result.

In [ ]:
# Define the variables we will be plotting, as well as labels and titles

# Plotting variables
xdata = x_h
ydata = logRate_h
dydata = dlogRate_h


# Construct the model for plotting and calculating residuals

# Find the x and y values for the model line
xmin = np.min(xdata)  # use the np.min function to find the smallest x-value
xmax = np.max(xdata)  # same for max
xpoints = np.linspace(xmin, xmax, 200)
ypoints = np.log(R0 * np.exp(-mu * xpoints) + Rb)  # R0 * e^{-mu*x} + Rb model

# Find the model for the residuals
ymodel = np.log(R0 * np.exp(-mu * xdata) + Rb)
residualsVec = ydata - ymodel

# Create the scatter plot with annotations

plt.figure(figsize=(9, 5))

# Create the main plots
plt.errorbar(
    xdata, ydata, dydata, marker='.', 
    linestyle='', label="Hypethetical data (Ln transformed)"
    )
plt.plot(
    xpoints, ypoints, "r-",
    label="Ln$(R_0 \, e^{-\mu x} + R_b)$ model using initial guesses"
    )
plt.xlabel("Shielding Thickness (x, number of plastic sheets)")
plt.ylabel("Ln(R)")
plt.legend(edgecolor='black')  
plt.tick_params(axis='both', which='major')
plt.grid(color='grey', linestyle='--', alpha=0.6)  # Add slightly darker grid

# Rb plateau annotation
plt.hlines(
    y=np.log(Rb), xmin=xmin - 1.2, xmax=xmax, color='black', linestyle='--'
    ) 

plt.text(
    xmin - 6, np.log(Rb), 'Ln(Rb) plateau', 
    bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.3'),
    horizontalalignment='center', verticalalignment='center'
    )  # arrow

arrow_x = xmin - 1.1  # Adjust this value to position the arrow
arrow_y = np.log(Rb)  # Adjust this value to position the arrow
plt.annotate(
    '', xy=(arrow_x, arrow_y), xytext=(arrow_x - 2., arrow_y),
    arrowprops=dict(facecolor='black', shrink=0.15),
    horizontalalignment='center', verticalalignment='center'
    )  # text box

plt.hlines(
    y=np.log(R0 + Rb), xmin=xmin - 1.2, xmax=xmin - 0.2,
    color='k', linestyle='--'
    )  

# Ln(R0+Rb) intercept annotation
arrow_x = xmin - 1.1  # Adjust this value to position the arrow
arrow_y = np.log(R0 + Rb)  # Adjust this value to position the arrow
plt.annotate(
    '', xy=(arrow_x, arrow_y), xytext=(arrow_x - 2, arrow_y),
    arrowprops=dict(facecolor='black', shrink=0.15),
    horizontalalignment='center', verticalalignment='center'
    )  # arrow
plt.text(
    xmin - 7.1, np.log(R0 + Rb), 'Ln(R0+Rb) intercept', 
    bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.3'),
    horizontalalignment='center', verticalalignment='center'
    )  # text box


# mu annotation
arrow_x = xmin + 7  # Adjust this value to position the arrow
arrow_y = np.log(Rb) + 1.6  # Adjust this value to position the arrow
plt.annotate(
    '', xy=(arrow_x, arrow_y), xytext=(arrow_x + 1.4, arrow_y + 0.4),
    arrowprops=dict(facecolor='black', shrink=0.15),
    horizontalalignment='center', verticalalignment='center'
    )  # arrow
plt.text(
    arrow_x + 3.9, arrow_y + 0.6, 'Slope = $-\mu$',
    bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.3'),
    horizontalalignment='center', verticalalignment='center'
    )  # text box

# Set x-axis limit
plt.xlim(-10, max(xdata) + 1)

plt.show()

# Determine residuals, plot residuals and calculate chi-squared

ymodel = np.log(R0 * np.exp(-mu * xdata) + Rb)
residualsVec = ydata - ymodel

plt.figure(figsize=(9, 5))
plt.errorbar(
    xdata, residualsVec, dydata, fmt="o",
    markersize=3, label="Residuals"
    )
plt.hlines(y=0, xmin=xmin, xmax=xmax, color='k')  # draw a black line at y = 0.
plt.xlabel(x_label)  # re-use the x_label from the scatter plot with model
plt.ylabel("Residual = data - model")
plt.grid(True)
plt.xlim(-10, max(xdata) + 1)
plt.legend(edgecolor='black')  
plt.show()

# Calculate chi-squared and output current fit parameter values
chi2 = np.sum((residualsVec / dydata)**2) / (len(residualsVec) - 3)
print("R0: ", R0, "counts/s")
print("mu: ", mu, "sheets^-1")
print("Rb: ", Rb, "counts/s")
print("Ln(R0+Rb): ", np.log(R0 + Rb))
print("Ln(Rb): ", np.log(Rb))
print("Weighted chi-squared: ", chi2)

### Your turn #5

Look at the scatter and residuals plots above. Which change to one of the parameters should have the highest impact in improving the fit?

##### **Answer**

It looks like the slope parameter (-μ) needs to be steeper, meaning that we need to increase μ. Let's try changing that next.

## 2.3 Adjusting the fit parameters to improve the fit of the $\text{Ln}(R_0 \, e^{-\mu x} + R_b)$ model

During Lab 10 we will introduce a new `fit_plot` widget to help with the process of minimizing chi-squared. However, for now, we will try increasing our initial guess of $\mu = 0.33 \text{ sheets}^{-1}$ to $\mu = 0.39 \text{ sheets}^{-1}$ to make the slope of that initial linear portion more steep. Notice that this improves our residuals and our chi-squared goes from 4.3 down to 1.6, so this is certainly an improvement in the model.

If you like, try adjusting the three fit parameters further to see if you can get an even better fit.

In [ ]:
# Update the fit parametes

# Left at initial guesses for now
R0 = 18  # counts/s
Rb = 0.5  # counts/s

# Updated from 0.33 to 0.39
mu = 0.39  # 1/sheets

# Plot the transformed data

xdata = x_h
ydata = logRate_h
dydata = dlogRate_h

# Find the x and y values for the model line
xmin = np.min(xdata)  # use the np.min function to find the smallest x-value
xmax = np.max(xdata)  # same for max
xpoints = np.linspace(xmin, xmax, 200)
ypoints = np.log(R0 * np.exp(-mu * xpoints) + Rb)  # Ln(R0 * e^{-mu*x} + Rb)

graph_title = "Linearized hypothetical count-rate data (semilog plot)"
x_label = "Shielding Thickness (x, number of plastic sheets)"
y_label = "Ln(R)"

plt.figure()
plt.errorbar(
    xdata, ydata, dydata, marker='.',
    linestyle='', label="Hypethetical data (Ln transformed)"
    )
plt.plot(
    xpoints, ypoints, "r-",
    label="Ln$(R_0 \, e^{-\mu x} + R_b)$ model using initial guesses"
    )
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.title(graph_title)
plt.legend()
plt.grid(True)
plt.show()

# Determine residuals and calculate chi-squared

ymodel = np.log(R0 * np.exp(-mu * xdata) + Rb)
residualsVec = ydata - ymodel

plt.figure()
plt.errorbar(
    xdata, residualsVec, dydata, fmt="o",
    markersize=3, label="Residuals"
    )
plt.hlines(y=0, xmin=xmin, xmax=xmax, color='k')  # draw a black line at y = 0.
plt.xlabel(x_label)  # re-use the x_label from the scatter plot with model
plt.ylabel("Residual = data - model")
plt.legend()
plt.grid(True)
plt.show()

# Calculate chi-squared and output current fit parameter values
chi2 = np.sum((residualsVec / dydata)**2) / (len(residualsVec) - 3)
print("R0: ", R0, "counts/s")
print("mu: ", mu, "sheets^-1")
print("Rb: ", Rb, "counts/s")
print("Ln(R0+Rb): ", np.log(R0 + Rb))
print("Ln(Rb): ", np.log(Rb))
print("Weighted chi-squared: ", chi2)

## Your turn #6

As usual, review the Lab 10 instructions and copy in any analysis code that you feel will be relevant. Some final notes:
* Recall that the data set used in this prelab was a hypothetical data set designed to give data with the same general behavour as your Lab 09 data, but where the fitting parameters we used in this prelab will not be the same as what you expect them to be with your Lab 09 data.
* You will be expected to add some additional data points to your data set during Lab 10 to help provide your model with better fit parameters.
* You will notice in the Lab 10 notebook that we include some code for the new `fit_plot` widget that we will be using to help with fitting the $\text{Ln}(R_0 \, e^{-\mu x} + R_b)$ model to our semilog data. 

# Submit

Steps for submission:

1. Click: Run => Run_All_Cells
2. Read through the notebook to ensure all the cells executed correctly and without error.
3. File => Save_and_Export_Notebook_As->HTML
4. Inspect your html file
5. Upload the HTML document to the lab submission assignment on Canvas.